# 环境配置


In [27]:
#获取IP地址和所属地区
!curl ipinfo.io/country
!curl ipecho.net/plain

JP
205.178.183.46

In [20]:
import datetime


# 时间转换器，同时将毫秒转为秒
def stamp_to_time(timestamp):
    timestamp_str = str(timestamp)
    if len(timestamp_str) == 13:  # 毫秒转秒
        timestamp1 = int(timestamp_str[0:10])
    else:
        timestamp1 = int(timestamp_str)
    dt_object = datetime.datetime.fromtimestamp(timestamp1)
    dt_string = dt_object.strftime("%Y-%m-%d %H:%M:%S")
    return dt_string

# 测试账户连接等基本功能


In [132]:
# 连接测试账户
from binance.client import Client

# import spot_function

# 输入你的 API key 和 secret key
api_key = "I0dZRDPJACmX8jDmVme3HWh3DJgC82aAoqpQYwfVPog9ZwheX4uFfMmPvqT3noh4"
api_secret = "t2wK9YE8nDM4scrHX0i7OSjxcDhizz5zMKelKNgnudFpg5mcrKhX43X9TQcsCOfX"

# 连接到测试网络

client = Client(api_key, api_secret, testnet=True)


# 获得账户信息
def all_position():
    account_info = client.get_account()

    for balance in account_info["balances"]:
        print(
            f"Asset: {balance['asset']}, Free: {balance['free']}, Locked: {balance['locked']}"
        )

    return account_info


account_info = all_position()

Asset: BNB, Free: 0.00000000, Locked: 0.00000000
Asset: BTC, Free: 0.00000000, Locked: 0.00000000
Asset: BUSD, Free: 10000.00000000, Locked: 0.00000000
Asset: ETH, Free: 0.00000000, Locked: 0.00000000
Asset: LTC, Free: 500.00000000, Locked: 0.00000000
Asset: TRX, Free: 500000.00000000, Locked: 0.00000000
Asset: USDT, Free: 327505.25680122, Locked: 0.00000000
Asset: XRP, Free: 50000.00000000, Locked: 0.00000000


In [ ]:
# 市价单
symbol = "BTCUSDT"
quantity = 1
params = {
    "symbol": "BTCUSDT",
    "side": "BUY",
    "type": "MARKET",
    "quantity": 0.1,
}

order = client.create_order(**params)
print(order)

In [141]:
# 限价单
#symbol, price, quantity, order_type
order = limit_order("BTCUSDT", 27106.01, 0.1, "IOC")

{'symbol': 'BTCUSDT', 'orderId': 13850674, 'orderListId': -1, 'clientOrderId': 'NGzWQvSxIoxlDr2JU4G4Nd', 'transactTime': 1685805759201, 'price': '27106.01000000', 'origQty': '0.10000000', 'executedQty': '0.00000000', 'cummulativeQuoteQty': '0.00000000', 'status': 'EXPIRED', 'timeInForce': 'IOC', 'type': 'LIMIT', 'side': 'BUY', 'workingTime': 1685805759201, 'fills': [], 'selfTradePreventionMode': 'NONE'}


In [144]:
# 检查订单状态
order_status = client.get_order(
    symbol = "BTCUSDT",
    orderId = order['orderId']
)
print(order_status)

{'symbol': 'BTCUSDT', 'orderId': 13850674, 'orderListId': -1, 'clientOrderId': 'NGzWQvSxIoxlDr2JU4G4Nd', 'price': '27106.01000000', 'origQty': '0.10000000', 'executedQty': '0.00000000', 'cummulativeQuoteQty': '0.00000000', 'status': 'EXPIRED', 'timeInForce': 'IOC', 'type': 'LIMIT', 'side': 'BUY', 'stopPrice': '0.00000000', 'icebergQty': '0.00000000', 'time': 1685805759201, 'updateTime': 1685805759201, 'isWorking': True, 'workingTime': 1685805759201, 'origQuoteOrderQty': '0.00000000', 'selfTradePreventionMode': 'NONE'}


In [129]:
# 清仓
order = clean_position()

市价单卖出 0.8 BTC, 成交金额 21731.36472671 USDT


In [ ]:
symbol = "BTCUSDT"
orders = client.get_all_orders(symbol=symbol)
for order in orders:
    print(order)

In [124]:
print_open_order()

In [80]:
# 清空某一symbol下的所有资产
account_info = client.get_account()

# 遍历所有的资产
for balance in account_info["balances"]:
    asset = balance["asset"]
    free = float(balance["free"])
    locked = float(balance["locked"])

    # 如果是 BTC 资产
    if asset == "BNB":
        # 如果有可用余额，则卖出并换成 USDT
        if free > 0:
            symbol = "BNBUSDT"
            order = client.order_market_sell(symbol=symbol, quantity=free)
            print(f"市价单卖出 {free} {asset}, 成交金额 {order['cummulativeQuoteQty']} USDT")
        # 如果有冻结余额，则先取消冻结
        if locked > 0:
            client.cancel_all_orders(symbol=symbol)
            print(f"取消 {asset} 的所有挂单")

# Websocket 连接
# 運行這一段code可以開啟程式交易，需要先將封裝Function內的def都執行一遍

In [225]:
# 測試ticker，买进后立刻卖出
import websocket
import json
from collections import deque
# from IPython.display import clear_output
import threading

last_diff = 0       #记录上一个窗口的diff，防止在大跌区间有人小量做多而触发交易
last_btc = 0
last_eth = 0
trade_flag = False  # 目前只挂一单，True代表已挂单
len_flag = False    # 记录deque的长度
account_info_inital = None #账户信息
order_book = []
sell_book = []

order_price = 0
order_time = 0
count = 0
queue_len = 12 

# 创建固定长度的双端队列
transaction_dequeue = deque(maxlen=queue_len)

def on_message(ws, message):
    global trade_flag
    data = json.loads(message)
    #print(data)
    
    eventtype = data["e"]
    symbol = data['s']
    if eventtype == "aggTrade":
        if trade_flag == False and symbol == "BTCUSDT":
            trade_fif(data)
        elif trade_flag == True and symbol == "ETHUSDT":
            trade_evaluate(data)

    elif eventtype == "24hrMiniTicker":
        # 交易Function, data, Buy_control, print_control
        deal_new(data, False, False)
    elif eventtype == "trade":
        deal_trade(data)
    

def on_error(ws, error):
    print(error)


def on_close(ws):
    print("### closed ###")


def on_open(ws):
    # 先清一次仓
    order = clean_position()
    # 获得初始资金，以便计算盈亏
    global account_info_inital
    account_info_inital = all_position()

    ws.send(btc_subscribe_msg)
    ws.send(eth_subscribe_msg)

url = "wss://stream.binance.com:443/ws"
url_testnet = "wss://testnet.binance.vision/ws"

ws = websocket.WebSocketApp(
    url, on_message=on_message, on_open=on_open, on_close=on_close
)
ws.run_forever()

流式输出内容被截断，只能显示最后 5000 行内容。
2023-06-04 03:05:58 27158.120 0.00273000
0.010
2023-06-04 03:05:58 27158.110 0.00824000
0.000
2023-06-04 03:05:59 27158.110 0.00041000
-0.010
2023-06-04 03:05:59 27158.120 0.08100000
0.010
2023-06-04 03:05:59 27158.120 0.00300000
0.000
2023-06-04 03:05:59 27158.110 0.00345000
-0.010
2023-06-04 03:05:59 27158.110 0.00252000
-0.010
2023-06-04 03:05:59 27158.120 0.00062000
0.000
2023-06-04 03:05:59 27158.120 0.00651000
0.010
2023-06-04 03:05:59 27158.120 0.00217000
0.010
2023-06-04 03:05:59 27158.110 0.00168000
-0.010
2023-06-04 03:06:00 27158.110 0.00158000
-0.010
2023-06-04 03:06:00 27158.120 0.00052000
0.010
2023-06-04 03:06:00 27158.120 0.00409000
0.010
2023-06-04 03:06:00 27158.110 0.00601000
-0.010
2023-06-04 03:06:00 27158.120 0.00211000
0.000
2023-06-04 03:06:00 27158.120 0.00236000
0.010
2023-06-04 03:06:00 27158.110 0.00263000
0.000
2023-06-04 03:06:00 27158.110 0.00050000
-0.010
2023-06-04 03:06:00 27158.120 0.00046000
0.000
2023-06-04 03:06:01 27158.

True

In [226]:
# 比较交易前后的资产
analyze(account_info_inital, order_book, sell_book)

Initial information of all positions:
Asset: BNB, Free: 0.00000000, Locked: 0.00000000
Asset: BTC, Free: 0.00000000, Locked: 0.00000000
Asset: BUSD, Free: 10000.00000000, Locked: 0.00000000
Asset: ETH, Free: 0.00000000, Locked: 0.00000000
Asset: LTC, Free: 500.00000000, Locked: 0.00000000
Asset: TRX, Free: 500000.00000000, Locked: 0.00000000
Asset: USDT, Free: 327504.47851872, Locked: 0.00000000
Asset: XRP, Free: 50000.00000000, Locked: 0.00000000
---------------------------------------------------
Final information of all positions:
Asset: BNB, Free: 0.00000000, Locked: 0.00000000
Asset: BTC, Free: 0.00000000, Locked: 0.00000000
Asset: BUSD, Free: 10000.00000000, Locked: 0.00000000
Asset: ETH, Free: 0.05000000, Locked: 0.00000000
Asset: LTC, Free: 500.00000000, Locked: 0.00000000
Asset: TRX, Free: 500000.00000000, Locked: 0.00000000
Asset: USDT, Free: 327409.83673842, Locked: 0.00000000
Asset: XRP, Free: 50000.00000000, Locked: 0.00000000
----------------------------------------------

In [227]:
# 计算每一笔交易的损失
print(len(order_book))
print(len(sell_book))
# rint(order_book[1]["fills"])
sum = 0
for index in range(0, len(sell_book)):
    fill = order_book[index]
    # print(len(fill))
    if len(fill) == 0:
        continue
    x = float(fill[0]["price"])
    fill = sell_book[index]
    y = float(fill[0]["price"])
    print("diff is :", x - y)
    sum += x - y
print("sum is", sum)

14
13
diff is : -0.2699999999999818
diff is : 0.029999999999972715
diff is : 0.040000000000190994
diff is : 0.7000000000000455
diff is : 0.009999999999990905
diff is : 0.09999999999990905
diff is : 0.06999999999993634
diff is : -0.22000000000002728
diff is : 0.03999999999996362
diff is : 0.029999999999972715
diff is : 0.009999999999990905
diff is : 0.009999999999990905
diff is : 0.029999999999972715
sum is 0.5799999999999272


# 封装 Function


## 比较15最新交易内的差价

In [224]:
#判断盈亏和交易时间的Function
def trade_evaluate(data):
    global sell_book
    global trade_flag
    global order_price 
    global order_time
    time = data['T']
    time_diff = int(time) - int(order_time)
    print("ETH", stamp_to_time(data['T']), data['p'][0:9], data['q'])
    diff = float(data['p'][0:9]) - float(order_price)
    print("ETH diff:", diff, "order_price:", order_price)
    if diff > 0.7:
        print("==========止盈=========")
        order = clean_position()
        sell_book.append(order['fills'])
        trade_flag = False
    elif diff > 0 and time_diff > 500:
        print("=======错过行业止盈=======")
        order = clean_position()
        sell_book.append(order['fills'])
        trade_flag = False
    elif diff < -0.7:
        print("==========止亏==========")
        order = clean_position()
        sell_book.append(order['fills'])
        trade_flag = False
    else:
        print("Holding---------", time_diff, diff)
        print("=============================")


def trade_fif(data):
    global transaction_dequeue
    global len_flag
    global trade_flag
    global order_book
    global order_time
    global order_price
    global last_diff
    transaction_dequeue.append(data)

    # 小于queue_len笔的交易不计入
    if len_flag == False:
        length = len(transaction_dequeue)
        if length >= queue_len:
            len_flag = True
        print("-------Waiting-------")
        print(stamp_to_time(data['T']), data['p'][0:9], data['q'])
    else :
        first_trade = transaction_dequeue[0]    #最旧的一笔资料
        last_trade = transaction_dequeue[-1]      #最新的一笔资料

        diff = float(last_trade['p'][0:9]) - float(first_trade['p'][0:9])
        print("=============================")
        print(stamp_to_time(data['T']), data['p'][0:9], data['q'])
        print("%.3f" % diff)
        
        if diff >= 0.25 and last_diff > -0.2:
            print("=============Order============")
            order = market_order("ETHUSDT", 0.05)    
            order_book.append(order['fills'])
            trade_flag = True
            
            order_time = order['transactTime']
            order_price = order['fills'][0]['price']
            print("Order ETH price", order_price)

            #避免短期内再次触发
            transaction_dequeue.clear()
            len_flag = False
        
        last_diff = diff
        #print("=============================")
        
    

## deal_trade 的交易 Function，目前是采用比较最新一笔 trade 和最新 tickers 价格，如果超过 diff 就触发交易。

## 6月更新，暂时启用，直接比较15笔最新交易内的差价


In [ ]:
# deal_trade的交易Function
# 把deal_aggtrade拆分成两个部分
# 第一个部分帮助获得最新价格，打印行情，计算差价
# 第二个部分帮助下单
# 这样一来，在持仓后依旧可以更新市价信息
# 事件类型，是否买入，交易对，时间/时间差，价格/入场价，最新BTC价格, 最新ETH价格, 价格差， 交易量

def deal_trade(data):
    global last_btc
    global last_eth
    global trade_flag_agg
    global buy_price
    global transactTime
    price = float(data["p"][0:9])
    time_ori = data["T"]
    time = stamp_to_time(data["T"])  # 采用较快的Trade Time，为币安服务器内容交易时间
    isbuyer = data["m"]  # True的话为主动卖

    if trade_flag_agg == True:
        time_diff = int(time_ori) - int(transactTime)
        sell_control = cal_profit(buy_price, time_diff)
        if sell_control == True:  # 卖出
            # print function
            deal_print("trade", 2, "BTC", time, 0, 0, 0, 0, 0)
            trade_flag_agg = False
        else:  # 继续持仓
            # print function
            deal_print("trade", 1, "BTC", time_diff, buy_price, 0, last_eth, 0, 0)
            trade_flag_agg = True
    else:
        symbol = data["s"]
        if symbol == "BTCUSDT":
            diff = price - last_btc
            # print function
            quantity = float(data["q"][0:5])
            deal_print(
                "trade", 0, "BTC", time, price, last_btc, last_eth, diff, quantity
            )

            # 下单
            if last_btc != 0 and quantity > 0.05 and diff >= 2:
                print("Agg OrderRRRR=")
                order = market_order("ETHUSDT", 0.1)
                transactTime = order["transactTime"]
                buy_price = float(order["fills"][0]["price"][0:9])
                order_book.append(order["fills"])
                trade_flag_agg = True

        elif symbol == "ETHUSDT":
            price = float(data["p"][0:9])
            # diff = price - last_eth

            # print("ETH Agg Time is :", time, "Price is ：", price, "Diff is : ", diff)
            # print("----------------------")

In [ ]:
# 把deal_aggtrade拆分成两个部分
# 第一个部分帮助获得最新价格，打印行情，计算差价
# 第二个部分帮助下单
# 这样一来，在持仓后依旧可以更新市价信息
# 事件类型，是否买入，交易对，时间/时间差，价格/入场价，最新BTC价格, 最新ETH价格, 价格差， 交易量
def deal_print(
    event, trade_status, symbol, time, price, last_btc, last_eth, diff, quantity
):
    # global trade_flag_agg
    if event == "aggTrade":
        if trade_status == 0:  # 未买入
            print("----------------------")
            print(
                symbol,
                "Agg Time is :",
                time[10:],
                "Price is :",
                price,
                "Quanity is :",
                quantity,
                "Diff is : ",
                diff,
            )
            print("BTC ticker :", last_btc, "ETH ticker is :", last_eth)
        elif trade_status == 1:  # 买入，未达到止盈止亏
            print("----------------------")
            print(
                time,
                "Hold, enter price:",
                price,
                " ETH current ticker price is :",
                last_eth,
            )
        elif trade_status == 2:  # 止盈止亏卖出
            print("============================")
            print("============================")
            order = clean_position()
            sell_book.append(order["fills"])
            print("============================")
            print("============================")


def deal_aggtrade_new(data):
    global last_btc
    global last_eth
    global trade_flag_agg
    global buy_price
    global transactTime
    price = float(data["p"][0:9])
    time_ori = data["T"]
    time = stamp_to_time(data["T"])  # 采用较快的Trade Time，为币安服务器内容交易时间

    if trade_flag_agg == True:
        time_diff = int(time_ori) - int(transactTime)
        sell_control = cal_profit(buy_price, time_diff)
        if sell_control == True:  # 卖出
            # print function
            deal_print("aggTrade", 2, "BTC", time, 0, 0, 0, 0, 0)
            trade_flag_agg = False
        else:  # 继续持仓
            # print function
            deal_print("aggTrade", 1, "BTC", time_diff, buy_price, 0, last_eth, 0, 0)
            trade_flag_agg = True
    else:
        symbol = data["s"]
        if symbol == "BTCUSDT":
            diff = price - last_btc
            # print function
            quantity = float(data["q"][0:5])
            deal_print(
                "aggTrade", 0, "BTC", time, price, last_btc, last_eth, diff, quantity
            )

            # 下单
            if last_btc != 0 and quantity > 0.05 and diff >= 2:
                print("Agg OrderRRRR=")
                order = market_order("ETHUSDT", 0.1)
                transactTime = order["transactTime"]
                buy_price = float(order["fills"][0]["price"][0:9])
                order_book.append(order["fills"])
                trade_flag_agg = True

        elif symbol == "ETHUSDT":
            price = float(data["p"][0:9])
            # diff = price - last_eth

            # print("ETH Agg Time is :", time, "Price is ：", price, "Diff is : ", diff)
            # print("----------------------")

## 其他功能性 Function


In [97]:
# 比较交易前后的所得
def analyze(account_info_inital, order_book, sell_book):
    print("Initial information of all positions:")
    for balance in account_info_inital["balances"]:
        print(
            f"Asset: {balance['asset']}, Free: {balance['free']}, Locked: {balance['locked']}"
        )
    print("---------------------------------------------------")
    print("Final information of all positions:")
    account_info_final = all_position()

    print("---------------------------------------------------")
    print("order book is :")
    for row in order_book:
        print(row)
        # print(type(row[0]['price']))
        # print(row["fills"])
    print("---------------------------------------------------")
    print("sell book is :")
    for row in sell_book:
        print(row)
        # print(row["fills"])

In [8]:
# deal的新版本，只更新last_btc、last_eth
def deal_new(data, buy_control, print_flag):
    global last_btc
    global last_eth

    if data["e"] == "24hrMiniTicker":
        symbol = data["s"]

        # time = stamp_to_time(data['E']) #采用较快的Trade Time，为币安服务器内容交易时间
        if symbol == "BTCUSDT":
            price = float(data["c"][0:9])

            last_btc = price
        elif symbol == "ETHUSDT":
            price = float(data["c"][0:9])

            last_eth = price

In [212]:
# 获得账户信息
def usdt_amount(symbol):
    account_info = client.get_account()

    for balance in account_info["balances"]:
        if balance["asset"] != symbol:
            continue
        else:
            # print(f"Asset: {balance['asset']}, Free: {balance['free']}, Locked: {balance['locked']}")
            return balance["free"]


# 清仓
def clean_position():
    account_info = client.get_account()

    # 遍历所有的资产
    for balance in account_info["balances"]:
        asset = balance["asset"]
        free = float(balance["free"])
        locked = float(balance["locked"])

        # 如果是 BTC 资产
        if asset == "ETH" or asset == "BTC":
            # 如果有可用余额，则卖出并换成 USDT
            if free > 0:
                symbol = asset + "USDT"
                order = client.order_market_sell(symbol=symbol, quantity=free)
                print(f"市价单卖出 {free} {asset}, 成交金额 {order['cummulativeQuoteQty']} USDT")
                return order
            # 如果有冻结余额，则先取消冻结
            if locked > 0:
                client.cancel_all_orders(symbol=symbol)
                print(f"取消 {asset} 的所有挂单")
    print("无BTC或ETH持仓")


# 市价单
def market_order(symbol, quantity):
    params = {
        'symbol': symbol,
        'side': 'BUY',
        'type': 'MARKET',
        'quantity': quantity,
    }

    order = client.create_order(**params)
    #print(order['fills'])
    return order
# 限价单
def limit_order(symbol, price, quantity, order_type):
    params = {
        "symbol": symbol,  # 交易对
        "side": "BUY",  # 买入或卖出
        "type": "LIMIT",  # 限价单类型
        "price": price,  # 限价单价格
        "quantity": quantity,  # 交易数量
        "timeInForce": order_type
    }

    order = client.create_order(**params)
    print(order)
    return order
# 获得实时ticker
def get_all_tickers(client, symbol_target):
    tickers = client.get_all_tickers()
    for index in tickers:
        symbol = index["symbol"]
        if symbol == symbol_target:
            return index["price"]
        else:
            continue
# 获得历史订单
def print_history_order():
    orders = client.get_all_orders(symbol="ETHUSDT", limit=100)
    for index in orders:
        print(index)
# 获得未成交订单
def print_open_order():
    orders = client.get_open_orders(
    symbol="BTCUSDT"
    )
    for order in orders:
        print(order)

In [176]:
# 更改websocket订阅内容


btc_subscribe_msg = json.dumps(
    {"method": "SUBSCRIBE", "params": ["btcusdt@aggTrade"], "id": 1}
)

eth_subscribe_msg = json.dumps(
    {"method": "SUBSCRIBE", "params": ["ethusdt@aggTrade"], "id": 2}
)

btc_unsubscribe_msg = json.dumps(
    {"method": "UNSUBSCRIBE", "params": ["btcusdt@aggTrade"], "id": 3}
)




# 测试Function

In [ ]:
data = {'e': 'aggTrade', 'E': 1685795403631, 's': 'BTCUSDT', 'a': 2644208988, 'p': '27163.73000000', 'q': '0.00116000', 'f': 3132715796, 'l': 3132715796, 'T': 1685795403630, 'm': False, 'M': True}
print(type(data))

In [56]:
from collections import deque
from queue import Queue
import time

# 定义测试次数
num_tests = 1000000
# 创建长度为15的双端队列
transaction_dequeue = deque(maxlen=15)
# 创建固定长度为15的队列
# transaction_queue = Queue(maxsize=15)

def plus(a,b):
   return a + b

# 测试 float 减法
start_time = time.time()
for i in range(num_tests):
    a = 1 
    b = 1
    ans = a + b
end_time = time.time()
float_time = end_time - start_time
print(f"dequeue时间为：{float_time:.6f} 秒")

# 测试 int 减法
start_time = time.time()
for i in range(num_tests):
    a = 1
    b = 1
    ans = plus(a,b)
end_time = time.time()
int_time = end_time - start_time
print(f"queue时间为：{int_time:.6f} 秒")

# 比较时间差异
if float_time > int_time:
    print(f"int 减法比 float 减法快 {float_time/int_time:.2f} 倍")
else:
    print(f"float 减法比 int 减法快 {int_time/float_time:.2f} 倍")


dequeue时间为：0.081229 秒
queue时间为：0.084165 秒
float 减法比 int 减法快 1.04 倍
